# Part 3: Tool Use

## ChatModel Tool Use

### Chat Model

In [1]:
import os
from langchain_openai import ChatOpenAI

AVALAI_BASE_URL = "https://api.avalai.ir/v1"
GPT_MODEL_NAME = "gpt-4o-mini"

gpt4o_chat = ChatOpenAI(model=GPT_MODEL_NAME,
                        base_url=AVALAI_BASE_URL,
                        api_key=os.environ["AVALAI_API_KEY"])

### Search as Tool

In [2]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults

def search_tool(query: str):
    """Searches the internet for the query and returns the top 3 results.
    Args:
        query: The search query.
    """
    return TavilySearchResults(max_results=3).invoke(query)

query = "Who is Milad Mohammadi?"
search_tool(query)

[{'url': 'https://en.wikipedia.org/wiki/Milad_Mohammadi',
  'content': 'Milad Mohammadi Keshmarzi, known as Milad Mohammadi (Persian: ميلاد محمدی کشمرزی; born 29 September 1993) is an Iranian professional footballer who plays for Persian Gulf Pro League club Persepolis and the Iranian national team. Milad Mohammadi joined Rah Ahan in the summer of 2014 with a five–year contract[4] and made his debut for them in the first fixture of the 2014–15 Iran Pro League against Esteghlal.[5] On 6 February 2016 Mohammadi signed a contract with Russian Premier League club Terek Grozny until 2019.[7][8] Milad made his debut for Terek Grozny as a substitute on 2 April 2016 in a 3–2 win over Anzhi Makhachkala.[9] Mohammadi scored his first goal for Terek and his first ever professional league goal on 29 April 2017 in a 5–2 victory against Ural Yekaterinburg.'},
 {'url': 'https://www.wikiwand.com/en/articles/Milad_Mohammadi',
  'content': 'Milad Mohammadi Keshmarzi, known as Milad Mohammadi (Persian: م

### Bind Model with Tools

In [4]:
tools = [search_tool]
llm_with_tools = gpt4o_chat.bind_tools(tools)

In [5]:
response = llm_with_tools.invoke("Hello. How are you?")
response

AIMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 65, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-089db732-bd11-4eb3-bae2-34c4ca6ab075-0', usage_metadata={'input_tokens': 65, 'output_tokens': 31, 'total_tokens': 96, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
response = llm_with_tools.invoke("Who is Milad Mohammadi?")
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IFHuYyqU6gUOIGmfOJFmpVxl', 'function': {'arguments': '{"query":"Milad Mohammadi"}', 'name': 'search_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 67, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6eee6e44-f205-4802-a092-c1480f31dc51-0', tool_calls=[{'name': 'search_tool', 'args': {'query': 'Milad Mohammadi'}, 'id': 'call_IFHuYyqU6gUOIGmfOJFmpVxl', 'type': 'tool_call'}], usage_metadata={'input_tokens': 67, 'output_tokens': 19, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio

### Manually Running Tool Calls

In [7]:
tool_call = response.additional_kwargs['tool_calls'][0]
tool_call

{'id': 'call_IFHuYyqU6gUOIGmfOJFmpVxl',
 'function': {'arguments': '{"query":"Milad Mohammadi"}',
  'name': 'search_tool'},
 'type': 'function'}

In [8]:
func_name = tool_call['function']['name']
func_args = tool_call['function']['arguments']

print("Function name:", func_name)
print("Function arguments:", func_args)

Function name: search_tool
Function arguments: {"query":"Milad Mohammadi"}


In [9]:
# NEVER DO THIS IN PRODUCTION! This is just for demonstration purposes.
eval(f"{func_name}(*{func_args})")

[{'url': 'https://www.collinsdictionary.com/us/dictionary/english/query',
  'content': 'A query is a question, especially one that you ask an organization, publication, or expert. If you have any queries about this insurance, please contact our call center. 2. transitive verb. If you query something, you check it by asking about it because you are not sure if it is correct.'},
 {'url': 'https://dictionary.cambridge.org/us/dictionary/learner-english/query',
  'content': 'QUERY meaning: 1. a question: 2. to ask questions in order to check that something is true or correct: . Learn more.'},
 {'url': 'https://www.dictionary.com/browse/query',
  'content': 'to mark (a manuscript, proof sheet, etc.) with a query.\nto ask questions of.\nOrigin of query\nOther words from query\nWords that may be confused with query\nWords Nearby query\nDictionary.com Unabridged\nBased on the Random House Unabridged Dictionary, © Random House, Inc. 2023\nHow to use query in a sentence\nWhen a person uses their 

In [10]:
functions_dict = {
    "search_tool": search_tool,
}
print(functions_dict[func_name](*{func_args}))

[{'url': 'https://dblp.org/pid/164/0815', 'content': "We've just launched a new service: our brand new dblp SPARQL query service. Read more about it in our latest blog post or try out some of the SPARQL queries linked on the dblp web pages below. ... Milad Mohammadi, Song Han, Tor M. Aamodt, William J. Dally: On-Demand Dynamic Branch Prediction. IEEE Comput. Archit. Lett. 14 (1): 50-53 (2015)"}, {'url': 'https://www.linkedin.com/in/milad-mohammadi-32999715', 'content': 'Others named Milad Mohammadi in United States. Milad Mohammadi Ph.D. candidate in Mechanical Engineering Kansas City, MO. Milad Sadat-Mohammadi, PhD Austin, TX. Milad Mohammadi'}, {'url': 'https://scholar.google.com/citations?user=yLr1ykoAAAAJ&hl=en', 'content': 'Milad Mohammadi. Software Engineer, Google. Verified email at alumni.stanford.edu - Homepage. Data Science Artificial Intelligence Compilers Distributed Computing Computer Architecture. Articles Cited by Public access Co-authors. Title. Sort. Sort by citations 

## Graph as a tool

### Import graph from file

In [11]:
from advanced_search_graph import get_advanced_search_graph

# Get the compiled graph
advanced_search_graph = get_advanced_search_graph()

user_input = {"user_query": "I'am new to baking, How to bake a chocolate cake?"}
final_state = advanced_search_graph.invoke(user_input)

---Node Query Rewrite---
Rewritten query: "I'm a beginner in baking. What are the steps to bake a chocolate cake?"
---Node Search Internet---
Search results:
[{'url': 'https://startcooking.com/chocolate-cake-for-beginners', 'content': 'Finally I have discovered the perfect chocolate cake recipe for beginner cooks. (Don’t make the frosting until the cake is baked, and stone cold.) Measure all the cake ingredients and set them aside: whenever i bake a cake there’s something wrong with it – sometimes too moist, extra egg,too much of baking soda, sometimes needs more cooking but if i bake it again it looks like all the butter is coming out of the cake,etc etc………but then the main problem with my cakes is that they are alwayz fail i.e.,they never rise up……………….do help!!!!!!!! This recipe will only work if you bake the cake in a regular oven. My recipe for chocolate cake ONLY uses baking soda.'}, {'url': 'https://www.allrecipes.com/recipe/17981/one-bowl-chocolate-cake-iii/', 'content': '"\nEd

### Graph as Tool

In [13]:
def advanced_search_tool(query: str) -> str:
    """
    A tool that performs an advanced search for cooking and baking based on the user's query.
    Args:    
        query (str): The user's query.
    """
    input_state = {
        "user_query": query,
        "rewritten_query": None,
        "search_results": None,
        "final_answer": None,
    }
    output_state = advanced_search_graph.invoke(input_state)
    return output_state["final_answer"]

### Bind ChatModel with Tool

In [15]:
llm_with_advanced_search = gpt4o_chat.bind_tools([advanced_search_tool, search_tool])

### System Prompt for Tool Use

In [20]:
from langchain_core.messages import HumanMessage, SystemMessage

system_prompt="""You are a helpful assistant AI assistant that have access to 'advanced_search_tool' for searching about cooking and baking. 
And 'search_tool' for general questions that need internet search."""
messages = [
    SystemMessage(system_prompt),
    HumanMessage("How to bake cake?"),
]

In [21]:
response = llm_with_advanced_search.invoke(messages)
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_NS7IZS2KJcbVaUged4O0FiHF', 'function': {'arguments': '{"query":"how to bake a cake"}', 'name': 'advanced_search_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 143, 'total_tokens': 163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f3927aa00d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1110a200-b8ad-4bff-83bd-311c45d73ef8-0', tool_calls=[{'name': 'advanced_search_tool', 'args': {'query': 'how to bake a cake'}, 'id': 'call_NS7IZS2KJcbVaUged4O0FiHF', 'type': 'tool_call'}], usage_metadata={'input_tokens': 143, 'output_tokens': 20, 'total_tokens': 163, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'ou

In [14]:
messages = [
    SystemMessage(system_prompt),
    HumanMessage("How to bake a chocolate cake? Search the internet for me."),
]

In [15]:
response = llm_with_advanced_search.invoke(messages)
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Se72yelfd4yvKBepZ2Tvfrwq', 'function': {'arguments': '{"query":"how to bake a chocolate cake recipe"}', 'name': 'advanced_search_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 102, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-356fb11e-540b-4b01-93e7-b6a4135b647f-0', tool_calls=[{'name': 'advanced_search_tool', 'args': {'query': 'how to bake a chocolate cake recipe'}, 'id': 'call_Se72yelfd4yvKBepZ2Tvfrwq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 102, 'output_tokens': 23, 'total_tokens': 125, 'input_token_details': 

### Manually Running Tool Calls

In [22]:
tool_call = response.additional_kwargs['tool_calls'][0]
tool_call

{'id': 'call_NS7IZS2KJcbVaUged4O0FiHF',
 'function': {'arguments': '{"query":"how to bake a cake"}',
  'name': 'advanced_search_tool'},
 'type': 'function'}

In [23]:
func_name = tool_call['function']['name']
func_args = tool_call['function']['arguments']

print("Function name:", func_name)
print("Function arguments:", func_args)

Function name: advanced_search_tool
Function arguments: {"query":"how to bake a cake"}


In [24]:
functions_dict = {
    "advanced_search_tool": advanced_search_tool,
}
print(functions_dict[func_name](*{func_args}))

---Node Query Rewrite---
Rewritten query: "how to bake a cake step by step"
---Node Search Internet---
Search results:
[{'url': 'https://testfoodkitchen.com/how-to-bake-a-simple-cake-step-by-step/', 'content': 'How to Bake a Cake Step by Step: 1) Preheat oven to 350 degrees Fahrenheit. Grease and flour a 9 by 13 inch baking pan. 2) In a large bowl, mix together eggs, sugar, vanilla extract, and salt. Add the dry ingredients and mix until everything is well combined.'}, {'url': 'https://testfoodkitchen.com/how-to-bake-a-cake-step-by-step/', 'content': 'How to Bake a Cake Step by Step: A Recipe for Success. Every baker has their own tricks and techniques they use to make the perfect cake. Here are a few tips to help you get started: Preheat your oven before beginning the baking process. This will help ensure that your cake cooks evenly and creates an even outcome.'}, {'url': 'https://bakingjoys.com/how-to-bake-a-cake-step-by-step/', 'content': "Step 2: How to bake a cake step by step and